In [ ]:
import math
from os import path

import astropy.units as u
import biff.scf as bscf
import gala.potential as gp
from gala.units import galactic
import h5py
import numpy as np
from scipy.optimize import minimize

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

from gala.potential.scf import compute_coeffs

In [ ]:
# hard-coded from Wang & Zhao (2012)
x0 = 1.49
y0 = 0.58
z0 = 0.4
q = 0.6

def dwek_density(x, y, z, src=math):
    r1 = (((x/x0)**2 + (y/y0)**2)**2 + (z/z0)**4) ** 0.25
    r2 = src.sqrt((q**2*(x**2+y**2) + z**2) / z0**2)
    return src.exp(-r1**2 / 2) + r2**-1.85*src.exp(-r2)

In [ ]:
nmax = 9
lmax = 19

S_filename = '../data/Sn{0}l{1}m.npy'.format(nmax,lmax)
Serr_filename = '../data/Sn{0}l{1}m_err.npy'.format(nmax,lmax)
if not path.exists(S_filename):
    S, Serr = compute_coeffs(dwek_density, nmax=nmax, lmax=lmax, 
                             M=1., r_s=1., progress=True, S_only=True)
    np.save(S_filename, S)
    np.save(Serr_filename, Serr)
    
S = np.load(S_filename)
Serr = np.load(Serr_filename)

In [ ]:
bar = bscf.SCFPotential(m=1, r_s=1.,
                        Snlm=S,
                        Tnlm=np.zeros_like(S),
                        units=galactic)

In [ ]:
grid = np.linspace(-4, 4, 128)

fig, axes = plt.subplots(1, 2, figsize=(12, 6))
# _ = bar.plot_density_contours(grid=(grid, grid, 0.), ax=ax, )
_ = bar.plot_contours(grid=(grid, grid, 0.), ax=axes[0])
_ = bar.plot_contours(grid=(grid, 0, grid), ax=axes[1])

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 5))

xyz = np.zeros((3, 1024)) + 1e-8
xyz[0] = np.logspace(-1, 2, xyz.shape[1])
ax.plot(xyz[0], bar.mass_enclosed(xyz))
ax.set_xscale('log')
ax.set_yscale('log')
ax.axvline(x0)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5), 
                         sharex=True, sharey=True)

for i in range(3):
    xyz = np.zeros((3, 1024)) + 1e-8
    # xyz[i] = np.linspace(1, 100, xyz.shape[1])
    xyz[i] = np.logspace(-1, 1, xyz.shape[1])
    
    dens = bar.density(xyz)
    true_dens = np.array([dwek_density(x, y, z) for x,y,z in xyz.T])

    ax = axes[i]
    ax.plot(xyz[i], true_dens)
    ax.plot(xyz[i], dens)

ax.set_xscale('log')
ax.set_yscale('log')

ax.set_ylim(1e-6, 100)

# ---

fig, axes = plt.subplots(1, 3, figsize=(15, 5), 
                         sharex=True, sharey=True)

for i in range(3):
    xyz = np.zeros((3, 1024)) + 1e-5
    xyz[i] = np.linspace(1, 100, xyz.shape[1])

    ax = axes[i]
    grad = bar.gradient(xyz)
    ax.plot(xyz[i], grad[0])
    ax.plot(xyz[i], grad[1])
    ax.plot(xyz[i], grad[2])

ax.set_xscale('log')
ax.set_yscale('log')

# ax.set_ylim(1e-6, 4)

---

Like Banik and Bovy, add bar model to MWPotential2014

In [ ]:
mw = gp.BovyMWPotential2014()

pot = gp.CCompositePotential()
pot['disk'] = mw['disk']
pot['halo'] = mw['halo']
pot['bar'] = bscf.SCFPotential(m=5e9 / 9, r_s=1., # MAGIC NUMBER: just believe
                               Snlm=S,
                               Tnlm=np.zeros_like(S),
                               units=galactic)

In [ ]:
xyz = np.zeros((3, 1024)) + 1e-8
xyz[0] = np.linspace(0, 30, xyz.shape[1])
    
plt.figure(figsize=(8, 5))
plt.plot(xyz[0], mw.circular_velocity(xyz).to_value(u.km/u.s))
plt.plot(xyz[0], pot.circular_velocity(xyz).to_value(u.km/u.s))
plt.xlim(0, 30)
plt.ylim(0, 300)

---

## Visualize surface density

In [ ]:
grid = np.linspace(-25, 25, 128)
x,y,z = np.meshgrid(grid, grid, grid)
xyz = np.stack((x, y, z))
dx = grid[1] - grid[0]

In [ ]:
dx = grid[1] - grid[0]

In [ ]:
dens = pot.density(xyz.reshape(3, -1))

In [ ]:
surf_dens = np.sum(dens.reshape(xyz.shape[1:]), axis=-1) * dx * u.kpc

In [ ]:
grid = np.linspace(-25, 25, 128)

fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ax.pcolormesh(xyz[0, ..., -1], xyz[1, ..., -1], surf_dens.T.value, 
              norm=mpl.colors.LogNorm())

# _ = pot.plot_density_contours(grid=(grid, grid, 0.), ax=ax, 
#                               levels=np.logspace(7.5, 10, 16),
#                               cmap='Spectral')